In [11]:
import pandas as pd 
import numpy as np


In [12]:
df = pd.read_csv("C:/Users/user.IBRAHIM-IK-SZHE/LLM_HPO/hyperopt-llm-project\data/raw_data/jena_weather_2004_2020.csv",parse_dates=True, index_col="Date Time")

In [13]:
df = df.sort_index()

In [14]:
# Define cutoff dates
cutoff_date_start = '2004-01-01 00:10:00'   # validation starts
cutoff_date_end = '2004-06-01 00:10:00'  # holdout starts

In [15]:
# extract 6 months from raw data 
df_cutt = df[cutoff_date_start:cutoff_date_end]

In [16]:
df_cutt.index

DatetimeIndex(['2004-01-01 00:10:00', '2004-01-01 00:20:00',
               '2004-01-01 00:30:00', '2004-01-01 00:40:00',
               '2004-01-01 00:50:00', '2004-01-01 01:00:00',
               '2004-01-01 01:10:00', '2004-01-01 01:20:00',
               '2004-01-01 01:30:00', '2004-01-01 01:40:00',
               ...
               '2004-05-31 22:40:00', '2004-05-31 22:50:00',
               '2004-05-31 23:00:00', '2004-05-31 23:10:00',
               '2004-05-31 23:20:00', '2004-05-31 23:30:00',
               '2004-05-31 23:40:00', '2004-05-31 23:50:00',
               '2004-06-01 00:00:00', '2004-06-01 00:10:00'],
              dtype='datetime64[ns]', name='Date Time', length=21889, freq=None)

In [17]:
#Split train /test
# train from Jan2004-Mai2004
# test from Mai 2004 to jun 2004
cutoff_date_eval = "2004-05-01 00:10:00"   # validation starts
# Train: before jun 2004
train_df = df_cutt[: cutoff_date_eval]

# Eval : Last month 
eval_df = df_cutt[cutoff_date_eval:]



print("Train shape:", train_df.shape)
print("Eval shape:", eval_df.shape)


Train shape: (17425, 21)
Eval shape: (4465, 21)


In [18]:
train_df.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m²),PAR (µmol/m²/s),max. PAR (µmol/m²/s),Tlog (degC),CO2 (ppm)
Date Time,,,,,,,,,,,,,,,,,,,,,
2004-01-01 00:10:00,996.82,-0.84,272.57,-1.23,97.2,5.74,5.58,0.16,3.49,5.60,...,2.53,3.63,26.15,0.0,0.0,0.0,0.0,0.0,8.57,NaN
2004-01-01 00:20:00,996.69,-0.78,272.64,-1.22,96.8,5.77,5.59,0.18,3.49,5.60,...,2.08,3.25,29.38,0.0,0.0,0.0,0.0,0.0,9.07,NaN
2004-01-01 00:30:00,996.69,-0.77,272.64,-1.29,96.3,5.77,5.56,0.21,3.48,5.58,...,2.09,3.75,19.44,0.0,0.0,0.0,0.0,0.0,9.27,NaN
2004-01-01 00:40:00,996.67,-0.77,272.65,-1.39,95.6,5.77,5.52,0.25,3.45,5.54,...,1.94,3.63,19.66,0.0,0.0,0.0,0.0,0.0,8.94,NaN
2004-01-01 00:50:00,996.52,-0.77,272.66,-1.44,95.2,5.77,5.49,0.28,3.44,5.51,...,1.28,2.25,21.53,0.0,0.0,0.0,0.0,0.0,8.46,NaN


In [19]:
eval_df.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),...,wv (m/s),max. wv (m/s),wd (deg),rain (mm),raining (s),SWDR (W/m²),PAR (µmol/m²/s),max. PAR (µmol/m²/s),Tlog (degC),CO2 (ppm)
Date Time,,,,,,,,,,,,,,,,,,,,,
2004-05-01 00:10:00,985.21,0.59,274.92,-0.63,91.5,6.38,5.83,0.54,3.69,5.92,...,3.44,5.25,249.6,0.0,490.0,0.0,0.0,0.0,9.00,NaN
2004-05-01 00:20:00,985.11,0.81,275.15,-0.74,89.3,6.48,5.79,0.69,3.66,5.87,...,3.42,6.63,247.9,0.0,600.0,0.0,0.0,0.0,8.70,NaN
2004-05-01 00:30:00,985.05,0.85,275.20,-0.64,89.7,6.50,5.83,0.67,3.69,5.92,...,3.45,5.25,257.3,0.0,600.0,0.0,0.0,0.0,8.25,NaN
2004-05-01 00:40:00,985.10,0.98,275.32,-0.62,89.0,6.56,5.84,0.72,3.69,5.92,...,4.14,7.88,257.5,0.0,600.0,0.0,0.0,0.0,7.92,NaN
2004-05-01 00:50:00,985.10,1.03,275.37,-0.60,88.8,6.58,5.84,0.74,3.70,5.93,...,3.93,7.50,256.1,0.0,600.0,0.0,0.0,0.0,8.34,NaN


In [ ]:
# Save splits
train_df.to_csv("C:/Users/user.IBRAHIM-IK-SZHE/hyperopt-llm-project/data/raw_data/train.csv", index=True)
eval_df.to_csv("C:/Users/user.IBRAHIM-IK-SZHE/hyperopt-llm-project/data/raw_data/eval.csv", index=True)
